# Clustering Conformations using Python Libraries


## Import dependencies

In [1]:
import numpy as np
import pickle
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list, to_tree, centroid, cut_tree,fcluster
from matplotlib import pyplot as plt
from helper import info, threshold_remove
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
from sklearn.cluster import AgglomerativeClustering

### Isolate lists with missing residues beyond a threshold

In [2]:
threshold_remove(10)
threshold_remove(7,[(32,43),(149,158)])

complete removed_indices: [0, 3, 4, 5, 6, 12, 16, 19, 20, 21, 22, 26, 27, 28, 29, 30, 31, 43, 45, 54, 55, 56, 57, 58, 59, 60, 61, 63, 66, 67, 68, 70, 81, 96, 99, 102, 103, 106, 107, 110, 112, 115, 116, 119, 120, 121, 126, 127, 152, 156, 158, 160, 161, 167, 169, 170, 171, 173, 174, 176, 177, 178, 179, 180, 183, 185, 196, 201, 202, 203, 204, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 231, 233, 243, 245, 247, 251, 253, 260, 261, 264, 265, 267, 268, 269, 272, 273, 274, 296, 297, 321, 322, 327, 334, 335, 336, 338, 339, 347, 351, 353, 354, 357, 358, 359, 360, 361, 367, 370, 371, 379, 381, 384, 385, 387, 391, 393, 398, 399, 400, 401, 402, 403, 404, 408, 414, 422, 424, 430, 431, 432, 435, 436, 437, 438, 439, 441, 442, 451, 452, 453, 454, 455, 457, 458, 460, 461, 464, 465, 466, 467, 468, 469, 472, 473, 474, 477, 480, 481, 482, 484, 485, 488, 489, 495, 498, 499, 500, 501, 502, 503, 506, 507, 508, 509, 510, 515, 517, 519, 520, 524, 525, 526, 529, 530]
segment removed_indices: [0,

## Load presorted classification based on annotations list

In [ ]:
#@ open files
with open("chains_list.var","rb") as chains_list_var:
   chains_list = pickle.load(chains_list_var)
   chains_list_var.close()
#print(f"chains_list: {chains_list}")
with open("structures/opened_active.var", "rb") as open_active_var:
    open_active_list = pickle.load(open_active_var)
    open_active_var.close()
with open("structures/closed_inactive.var", "rb") as closed_inactive_var:
    closed_inactive_list = pickle.load(closed_inactive_var)
    closed_inactive_var.close()
with open("structures/opened_inactive.var", "rb") as open_inactive_var:
    open_inactive_list = pickle.load(open_inactive_var)
    open_inactive_var.close()

print(f"open_active: {len(open_active_list)}")
print(f"closed_inactive: {len(closed_inactive_list)}")
print(f"open_inactive: {len(open_inactive_list)}")
annotated_dict_list_codes= {"open_active": open_active_list, "closed_inactive": closed_inactive_list, "open_inactive": open_inactive_list} #dictionary of codes list
annotated_dict_list ={"open_active": list(), "closed_inactive": list(), "open_inactive": list()} #dictionary of list of indices
for i,conformation in enumerate(chains_list):
    for j,l in enumerate(annotated_dict_list_codes):
        if conformation in annotated_dict_list_codes[l]:
            #print(f"l: {l}")
            annotated_dict_list[l].append(i)
#print(f"annotated_dict_list: {annotated_dict_list_codes}")


### Open calculated rms matrix and chains list
#### matrix.var
Calculated rms matrix from aligning all of the choice conformation chosen by best_align.py and calculated by pml_script_all.py.
#### chains_list.var
Generated by reading through the dictionary of annotated conformations and making a list of chains of the conformation. This will give the labeling order of the rms matrix obtained since pml_script_all.py iterated through this list

In [ ]:
with open("matrix.var", "rb") as matrix_var:
   matrix = (pickle.load(matrix_var))
   matrix_var.close()
#print(f"matrix: {matrix}")
with open("matrix_AB.var", "rb") as matrix_AB_var:
   matrix_AB = (pickle.load(matrix_AB_var))
   matrix_AB_var.close()
with open("matrix_seg.var", "rb") as matrix_seg_var:
   matrix_seg = pickle.load(matrix_seg_var)
   matrix_seg_var.close()

### Ward's Algorithm
A linkage with minimum variance method. We define 
$$
T = |v| + |s| + |t|
$$
As the cardinality the forest _u_ and _v_ combined
Given the distance function is computed recursively with the equation:
$$
d[i][j] = d(i,j) = \sqrt{\frac{|v| + |s|}{T}d(v,s)^2 + \frac{|v| + |t|}{T}d(v,t)^2 - \frac{|v|}{T}d(s,t)^2}
$$
The merging cost, of which ward's algorithm tries to minimize the growth is defined as follows:
$$
Cost(u,v) = \frac{|u||v|}{|u| +|v|}||c_u - c_v ||^2
$$
$c_i$ is the vector of the centroid. And the derivation comes from the distance sum union of the two sets together minus the distance sum of the individual sets. 
<br>
It considers the case of merging and then calculates the variance from the centroid of the final merged cluster as the distance

## Info function:
 ```info(matrix,title,chains_list,annotated_dict_list,complete,kernel="linear",hierarchy_method = None, no_clusters=3, dist_metric = "euclidean", tsne= False, auto_cut_tree = True)```
### Usage:
### __Required Parameters__:
#### matrix:
    A matrix of values where each row represents a conformation and the features along the columns
    In this case we have RMSD of all the CA, RMSD of all CA + CB, RSMD of all CA of specific segments
#### title: 
    Title of the matrix for plotting purposes
#### chains_list:
    List chains/ conformations in the same order as the matrix
#### annotated_dict_list:
    Dictionary where each key is group classified, found by pdb file annotation and value is a list of indices of the conformations on the matrix
#### complete:
    For the purposes of identifying the coincidence of misclassification and missing residues beyond a certain threshold. Complete is a boolean, if True it will compare the coincidences to the full conformations which had missing residues more than a threshold (default 10). Else false it will compare to the conformation on specific segments which had missing more than a threshold (default 7).
### __Optional Parameters__:
#### kernel:
    For the use of PCA, Kernel method of PCA (if in use)
#### Hierarchy_method:
    Method of hierachical clustering, default is None. set to None then the function will use the graph for Kmeans clustering by default. Uses sklearn and scipy hierarchy clustering. 
    Choices: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘cosine’, ‘precomputed’}
#### no_clusters:
    Number of clusters to be found. However, the statistics will give an error if the amount of clusters and the length of the ```annotated_dict_list``` do not match
#### dist_metric:
    Distance metric for hierarchical clustering, by default euclidean
#### tsne:
    Data is visualized in 2 different ways T-SNE and PCA, By default tsne = False means PCA is used. Data is projected into 2 dimensions
#### auto_cut_tree:
    By default the tree will be automatically cut by the number of clusters, however, if set to False, manual cutting will ask the user to get left or right tree in a sequence from top down.
    The first choice will prompt the user for the leftmost tree and then the second choice for the middle tree.
    Use the calculated dendrogram to decide on the manual tree partitioning.
    Use: "left", "l", "right", or "r"
    The statistics will give an erroneous result if the middle tree, the second tree is chosen not to the immediate right of the first tree.



## Experiment 1:
Complete CA, T-SNE projection with Ward's algorithm to obtain 3 clusters.

In [ ]:
info(matrix,"Complete CA", chains_list,annotated_dict_list,complete = True, kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=True)

## Experiment 2:
Complete CA, PCA projection with Ward's algorithm to obtain 3 clusters.
### Analysis:
The data shows that open inactive is often misrepresented as closed in active 96 False negative and only 16 True positive, also T-sne gives a better cluster view of the data than PCA, which shades clusters as less distinct.

In [ ]:
info(matrix,"Complete CA", chains_list,annotated_dict_list,complete = True,kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=False)

### Experiment 3
Complete CA and CB RSMD Matrix, T-SNE projection with Ward's algorithm to obtain 3 clusters.
### Analysis:

In [ ]:
info(matrix_AB,"Complete CA and CB", chains_list,annotated_dict_list,complete = True,kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=True)

### Experiment 4
Complete CA and CB RSMD Matrix, PCA projection with Ward's algorithm to obtain 3 clusters.
### Analysis:

In [ ]:
info(matrix_AB,"Complete CA and CB", chains_list,annotated_dict_list,complete = True,kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=False)

### Experiment 5
Selected Segments 33 to 44, 150 to 159, found from previous pymol modelling analysis. RSMD Matrix, T-SNE projection with Ward's algorithm to obtain 3 clusters.

### Analysis:

In [ ]:
info(matrix_seg,"Complete CA and CB", chains_list,annotated_dict_list,complete = True,kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=True)

### Experiment 6
Selected Segments 33 to 44, 150 to 159, found from previous pymol modelling analysis. T-SNE projection with Ward's algorithm to obtain 3 clusters, Manual Tree parititioning, Cluster1 = Tree.left.left, Cluster2 = Tree.left.right, Cluster3 = Tree.right
### Analysis:
This gives quite a high F1 score, however the partitioning is not as intuitive since Cluster 1 and Cluster 2 are highly similar and have low cophenetic distances compared to the Cluster 3. Maybe a change of distance metric might give different clustering results.

Actually, running a script to check how many chains are missing more than 7 residues in on of the segments (keeping in mind that the segments are only 12 and 10 residues long). The script outputs 176 samples. This is more than the amount misclassified at 79 conformations.

Taking from the misclassified examples 2BHH_A on the top right of the graph of the misclassified conformation has a gap from residue 36 to 43 and 148 to 162 which given only two residues of course would have trouble identifying the clustering.
1AQ1_A missing from 36 to 44 and 148 to 160.

On the other side of the misclassified cluster, 2AOC_X has only missing 37-40, 4 key residues missing

In [ ]:
info(matrix_seg,"Complete CA and CB", chains_list,annotated_dict_list,complete = True,hierarchy_method = "ward", no_clusters=3,tsne=True,auto_cut_tree=False)